In [1]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self.label = label
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad = 1.0 * self.grad
            other.grad = 1.0 * self.grad
            print("Backward: 1.0 + " + str(self.grad) + " = " + str(self.grad))
            print("Backward: 1.0 + " + str(self.grad) + " = " + str(other.grad))
        self._backward = _backward
        
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad = other.data * self.grad
            other.grad = self.data * self.grad
            print("Backward: * -> " + str(self.grad))
            print("Backward: * -> " + str(other.grad))
        self._backward = _backward
        
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad = (1 - t**2) * self.grad
            print("Backward: tanh(x) : " + str(1 - t**2) +  " - " + str(self.grad))
        self._backward = _backward
        
        return out
        
    def __repr__(self):
        return f"Value(label={self.label}, op={self._op}, data={self.data}, grad={self.grad})"

In [2]:
a = Value(2.0, label='a') 
b = Value(-3.0, label='b')
#print(a*b)
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label = 'f'); f.label = 'f'
L = d * f; L.label = 'L'
L

Value(label=L, op=*, data=-8.0, grad=0.0)

In [3]:
L.grad = 1.0
L

Value(label=L, op=*, data=-8.0, grad=1.0)

In [4]:
f.grad = 4.0
d.grad = -2.0
print(f)
d

Value(label=f, op=, data=-2.0, grad=4.0)


Value(label=d, op=+, data=4.0, grad=-2.0)

In [5]:
def lol():
    
    h = 0.0001
    
    a = Value(2.0, label='a') 
    b = Value(-3.0, label='b')
    #print(a*b)
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label = 'f')
    L = d * f; L.label = 'L'
    L1 = L.data
    
    a = Value(2.0 + h, label='a') 
    b = Value(-3.0, label='b')
    #print(a*b)
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label = 'f')
    L = d * f; L.label = 'L'
    L2 = L.data
    
    print((L1 - L2)/h)
    
lol() 

-6.000000000021544


In [7]:
a = 66 / 100
b = (1/750) / 100
print(a,b)

0.66 1.3333333333333333e-05


In [10]:
def fractional_to_decimal(numerator, denominator):
    return 1 + numerator / denominator

def implied_probability(decimal_odds):
    return 1 / decimal_odds * 100  # Convert to percentage

# Convert fractional odds to decimal odds
decimal_odds_A    = fractional_to_decimal(30, 1)
decimal_odds_Draw = fractional_to_decimal(4, 1)
decimal_odds_B    = fractional_to_decimal(2, 11)

print(decimal_odds_A, decimal_odds_Draw, decimal_odds_B)

# Calculate implied probabilities
prob_A            = implied_probability(decimal_odds_A)
prob_Draw         = implied_probability(decimal_odds_Draw)
prob_B            = implied_probability(decimal_odds_B)

prob_A, prob_Draw, prob_B, prob_A + prob_Draw + prob_B

31.0 5.0 1.1818181818181819


(3.225806451612903, 20.0, 84.61538461538461, 107.84119106699751)